In [58]:
import requests
import requests
from bs4 import BeautifulSoup

200

In [61]:
url = "http://www.pixiv.net/ranking.php"
response = requests.get(url)
headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36', 
          'password': 'rd.)T6vF',
          'pixiv_id' : 'flicfloclol@hotmail.fr'}
response = requests.get(url, headers=headers, timeout = 10)

response.status_code

soup = BeautifulSoup(response.text)
links = []

for link in soup.find_all('a', class_="work _work"):
    links.append(link.get('href').split('/')[2])
    
for link in soup.find_all('a', class_="work _work multiple"):
    links.append(link.get('href').split('/')[2])
    
print(links)



['89070034', '89071274', '89068934', '89089711', '89068978', '89109614', '89068988', '89072784', '89094862', '89095933', '89068898', '89075330', '89072215', '89110432', '89069000', '89069060', '89069757', '89107668', '89113282', '89095580', '89069891', '89107270', '89081191', '89092386', '89074522', '89068901', '89077477', '89081000', '89086580', '89084248', '89096137', '89068921', '89086852', '89081707', '89068903', '89101579', '89086359', '89069792', '89092100', '89076404', '89081506']


In [64]:
#url = "https://www.pixiv.net/ajax/user/103410/illusts?ids%5B%5D=" + links[0]
url = "https://www.pixiv.net/en/artworks/89070034"
response = requests.get(url)
response = requests.get(url, headers=headers, timeout = 7000)



print(response.status_code)

soup = BeautifulSoup(response.text)
print(soup.get_text())
for link in soup.find_all('figcaption'):
    print(link)


200
#綾波レイ 綾波 - スコッティのイラスト - pixiv






